# NSDUH Drug Sequence Analysis Part 2:  Clustering
## Matthew J. Beattie
## University of Oklahoma
__November 23, 2021__

This notebook examines drug use pathways with clustering.

## Clustering the drug use pathways
In the previous notebook, we stored drug use pathways as adjacency matrices of weighted directed graphs.  In this notebook, we seek to find groupings of similar pathways.  We will do this using _k-means clustering_ as implemented by SciKit Learn Extra.  We were limited to this choice due to the memory required to store the distance matrix during the SciKit _k-medoids_ method.

There are 9 drugs that we consider in the pathways, most of which have an age-of-first-use recorded value.  Several, including prescription opioid abuse, can have imputed values.  For each respondent, we create a 1x10 vector (_AFUVECT_), each position of which is the AFU for the drug (plus one for 'start').  We then cluster these vectors.

### No null pathways
In previous experiments, the cluster that had _no substance use_ dominated the other clusters.  When the null pathway is included as a cluster, it acts as a catch-all for any pathways that don't begin with common substances.  Consequently, this experiment only considers respondents who have had some lifetime substance use.  

In [4]:
"""
Import python modules
"""
import pandas as pd
import numpy as np
import copy
import os
import pathlib, itertools
import time as timelib
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
#from sklearn.impute import KNNImputer
#from sklearn.preprocessing import StandardScaler
import random
from sklearn_extra import cluster as cs
from sklearn.cluster import KMeans
import pickle
import json
import pathutils as pu
import scipy.stats as stats
from math import dist
import mlflow
import mlflow.sklearn

HOME_DIR = pathlib.Path.home()
CW_DIR = pathlib.Path.cwd()

FIGW = 12
FIGH = 5
FONTSIZE = 8
FIGURESIZE = (FIGW,FIGH)

plt.rcParams['figure.figsize'] = (FIGW, FIGH)
plt.rcParams['font.size'] = FONTSIZE

plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE

random.seed(660806)


### Set experiment parameters
The experiment parameters are the year of the survey (unless it is a block of years), the number of clusters, and the name of the JSON dictionary file.  Determining the best number of K-Medoid clusters is computationally complex and too slow in a notebook on a PC or cloud VM.  We run an experiment that seeks to minimize inertia based upon number of clusters and maximum interations.  We rerun the K-means algorithm and change the number of clusters.  We then plot the inertia versus the number of clusters and look for an elbow in the curve.

The parameter _readpickle_ tells the script whether to calculate the clusters or accept them from a prior run.

In [2]:
# Set parameters
datapath = 'C:/Users/mjbea/OneDrive/GitHub/abuse_sequence/Data3/'
workingpath = 'C:/Users/mjbea/OneDrive/GitHub/abuse_sequence/Code3/'
outpath = 'C:/Users/mjbea/OneDrive/GitHub/abuse_sequence/Output3/'
year = '2016_2017_2018_2019'
jsondict = datapath + 'NSDUH_field.json'
readpickle = False


### Import Data
Read in NSDUH data using _pathutils.py_ user-defined functions.  Data read in includes:
* NSDUH data from the prior script, _step1_sequence_dataprep_multiyear.ipynb_ 
* Dictionaries of drug names and their reference indices
* JSON file containing the reference indices and names for values of the NSDUH variables
* _Demographics_ are defined as the set of NSDUH variables for comparisons

The data is then stored as two dataframes, each of which uses the NSDUH respondent ID (_RESPID_) as a key value.  One dataframe, _dfclust_ includes the RESPID and pathway data.  The second, _dfdemog_, includes the RESPID and demographic information.

In [3]:
"""
Read in NSDUH data into a pandas dataframe
"""
# Read datafiles
loadfname = datapath + 'NSDUH_' + str(year) + '_pathways.pkl'
nsduh2 = pd.read_pickle(loadfname)

# Drug name and indices are called by a user-defined function.
ident, rawafuvals, afuvals, drugnames, drugorder, drugnums, drugposition, startdemog, demographics = pu.surveyvars(year)

# These are the demographic column names from NSDUH (or SVCFLAG for SERVICE)
#demographics = ['CATAG6','SVCFLAG','IRSEX','IRMARIT','NEWRACE2','EDUHIGHCAT','IRWRKSTAT','GOVTPROG','INCOME','COUTYP4','AIIND102','RESPID']

# Decode dictionaries for NSDUH variables:
f1 = open(jsondict, 'r')
nsduhDecoder = json.load(f1)
f1.close()
nsduh2.head()

# Add the yearly average sample weight to the demographics list
demographics.remove('ANALWT_C')
demographics.append('YRWEIGHT')

In [4]:
nsduh2[nsduh2['RESPID']=='201953777216']

,IRALCAGE,IRMJAGE,IRHERAGE,IRHALLUCAGE,IRINHALAGE,IRMETHAMAGE,AGE2,CATAG6,IRSEX,IRMARIT,...,AIIND102,RESPID,SVCFLAG,IRTOBAGE,IRCOC2AGE,YRWEIGHT,IRSCRIPAGE,AFUPATH,AFUVECT,UNWAFUPATH


In [5]:
# Separate the dataset into two smaller ones to work with
dfdemog = nsduh2[demographics]
dfdemog.head()

,AGE2,SVCFLAG,CATAG6,IRSEX,IRMARIT,NEWRACE2,EDUHIGHCAT,IRWRKSTAT,GOVTPROG,INCOME,COUTYP4,AIIND102,RESPID,YRWEIGHT
0,14,0,3,2,1,1,4,4,2,4,3,2,201611635143.0,204.858562
2,15,0,4,1,1,7,1,3,2,2,1,2,201635755143.0,2533.458396
3,17,0,6,2,1,1,3,2,2,3,1,2,201692675143.0,6203.973093
4,13,0,3,1,1,5,4,4,2,2,2,2,201659596143.0,1386.672703
5,16,0,5,1,2,1,2,4,2,2,2,2,201641106143.0,2384.841656


In [8]:
dfclust = nsduh2[['RESPID','AFUVECT']]
dfclust.head()

,RESPID,AFUVECT
0,201611635143.0,"[0, 16, 15, 20, 991, 991, 991, 991, 991, 991]"
2,201635755143.0,"[0, 26, 16, 991, 991, 991, 991, 991, 991, 991]"
3,201692675143.0,"[0, 5, 18, 32, 34, 991, 991, 991, 991, 991]"
4,201659596143.0,"[0, 991, 14, 991, 991, 991, 991, 991, 991, 991]"
5,201641106143.0,"[0, 991, 991, 991, 991, 991, 991, 991, 991, 991]"


In [9]:
# Merge the dfclust and dfdemog dataframes to match the observations and split back into
# final dfclust and dfdemog.  This avoids NA values due to a RESPID mismatch.
merged_total = pd.merge(dfclust, dfdemog, on=['RESPID'])
dfclust = merged_total[['RESPID','AFUVECT','YRWEIGHT']]
dfdemog = merged_total[['RESPID', 'CATAG6', 'SVCFLAG', 'IRSEX', 'IRMARIT', 'NEWRACE2', 'EDUHIGHCAT', 'IRWRKSTAT',
       'GOVTPROG', 'INCOME', 'COUTYP4', 'AIIND102','YRWEIGHT']]


### Clustering
K-Medoids fails because the SKLearn version requires storing the entire n x n distance matrix into memory.  The 100k+ x 100k+ matrix is too large to do that.  So we run KNN instead.  We set the number of clusters at 16 based upon runs from one year of data.

### Simplify Cluster Definition
K-Means is used for clustering because the SKLearn implementation can run without storing the inter-observation distance matrix into memory.  However, the centers of the clusters are difficult to interpret because they are _means_.  We try to simplify the definitions using two methods.  First, we round the cluster centers to integers.  Second, we find the _medoid_ of the clusters.  These values are written to file out.

In [14]:
# Create an array of the PATHVECT values for input into SciKit KMeans()
patharray = np.array(list(dfclust['AFUVECT']))
weights = np.array(list(dfclust['YRWEIGHT']))

# Initialize mlflow and run experiment
#clustrange = [1,2,3,4,5,6,7,8,9,10]
#clustrange = [11,12,13,14,15,16,17,18,19,20]
clustrange = [21,22,23,24,25,26,27,28,29,30]
n_init = 20
max_iter = 1000
tol = 0.0001

# Setup mlflow experiment
experiment_name = "Kmeans Clusters from 21-30"
mlflow.set_experiment(experiment_name)

for i in clustrange:
    n_clusters = i

    # Setup filenames
    descout = outpath + 'Kmeans_' + str(n_clusters) + 'clust_' + str(year) + '_nonullpath_clustdesc.txt'
    modpkl = workingpath + 'Kmeans_' + str(n_clusters) + 'clust_' + str(year) + '_nonullpath_model.pkl'
    clustpkl = workingpath + 'Kmeans_' + str(n_clusters) + 'clust_' + str(year) + '_nonullpath_clust.pkl'
    demogpkl = workingpath + 'Kmeans_' + str(n_clusters) + 'clust_' + str(year) + '_nonullpath_demog.pkl'

    with mlflow.start_run():
        # Calculate clusters and save model to a pickle file (or load model)
        if readpickle != True:
            model = KMeans(n_clusters=n_clusters, init='k-means++', n_init=n_init, max_iter=max_iter, 
                           tol=tol, verbose=0, random_state=None, copy_x=True, algorithm='auto')
            model.fit(patharray, sample_weight=weights)
            pickle.dump(model, open(modpkl, 'wb'))
        else:
            model = pickle.load(open(modpkl, 'rb'))

        pred = model.predict(patharray)

        # Append labels to the dfdemog dataset
        dfdemog['labels'] = model.labels_
        dfclust['labels'] = model.labels_

        # Store parameters and metrics into mlflow
        mlflow.log_param("n_clusters", n_clusters)
        mlflow.log_param("n_init", n_init)
        mlflow.log_param("max_iter", max_iter)
        mlflow.log_param("tol", tol)
        mlflow.log_metric("inertia", model.inertia_)

        # Generate medoids of clusters
        medoids = []
        for j in range(0, len(model.cluster_centers_)):
            clustcenter = model.cluster_centers_[j]
            clust = dfclust[dfclust['labels']==j]
            clust['distfrommean'] = clust.apply(lambda row: dist(np.array(row['AFUVECT']), clustcenter), axis=1)
            medoids.append(clust.loc[clust['distfrommean'].idxmin()]['AFUVECT'])
        medoids = np.array(medoids)

        # Print cluster descriptions to a file
        f = open(descout, 'w')
        print('\n******\nDRUG VECTOR POSITIONS\n*****\n', file=f)
        print(drugnums, file=f)
        print('\n******\nCLUSTER CENTERS\n******\n', file=f)
        print(model.cluster_centers_, file=f)
        print('\n******\n INTEGER CLUSTER CENTERS\n******\n', file=f)
        print(np.array(model.cluster_centers_, dtype='int'), file=f)
        print('\n******\n CLUSTER MEDOIDS\n******\n', file=f)
        print(medoids, file=f)
        print('\n******\nCLUSTER ITERATIONS\n******\n', file=f)
        print(model.n_iter_, file=f)
        print('\n******\nCLUSTER INERTIA\n******\n', file=f)
        print(model.inertia_, file=f)
        print('\n*****\nFRACTION OF POPULATION FOR EACH CLUSTER\n*****\n', file=f)
        totweight = clustpkl['YRWEIGHT'].sum()
        clustfracs = clustpkl.groupby(['labels']).sum('YRWEIGHT')/totweight
        print(clustfracs, file=f)
        f.close()

        mlflow.log_artifact(descout)
        
        # Get json file with variable definitions
        f1 = open(jsondict, 'r')
        nsduhDecoder = json.load(f1)
        f1.close()

        ident, rawafuvals, afuvals, drugnames, drugorder, drugnums, drugposition, startdemog, demographics = pu.surveyvars(year)

        # Save datasets to pickle files for future use
        dfclust.to_pickle(clustpkl)
        mlflow.log_artifact(clustpkl)
        dfdemog.to_pickle(demogpkl)
        mlflow.log_artifact(demogpkl)
        mlflow.log_artifact(modpkl)

    # End mlflow run
    mlflow.end_run()        


C:\Users\mjbea\anaconda3\envs\py38\lib\site-packages\sklearn\cluster\_kmeans.py:786: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  warnings.warn("'precompute_distances' was deprecated in version "
C:\Users\mjbea\anaconda3\envs\py38\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
C:\Users\mjbea\anaconda3\envs\py38\lib\site-packages\sklearn\cluster\_kmeans.py:786: FutureWarning: 'precompute_distances' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25). It has no effect
  warnings.warn("'precompute_distances' was deprecated in version "
C:\Users\mjbea\anaconda3\envs\p

In [7]:
clustpkl = pd.read_pickle(workingpath + 'Kmeans_11clust_2016_2017_2018_2019_nonullpath_clust.pkl')

In [8]:
clustpkl.head()

,RESPID,AFUVECT,YRWEIGHT,labels
0,201611635143.0,"[0, 16, 15, 20, 991, 991, 991, 991, 991, 991]",204.858562,3
1,201635755143.0,"[0, 26, 16, 991, 991, 991, 991, 991, 991, 991]",2533.458396,0
2,201692675143.0,"[0, 5, 18, 32, 34, 991, 991, 991, 991, 991]",6203.973093,10
3,201659596143.0,"[0, 991, 14, 991, 991, 991, 991, 991, 991, 991]",1386.672703,2
4,201641106143.0,"[0, 991, 991, 991, 991, 991, 991, 991, 991, 991]",2384.841656,5


In [13]:
print('Fraction of population for each cluster')
totweight = clustpkl['YRWEIGHT'].sum()
clustfracs = clustpkl.groupby(['labels']).sum('YRWEIGHT')/totweight
print(clustfracs)

Fraction of population for each cluster
        YRWEIGHT
labels          
0       0.209984
1       0.047161
2       0.173413
3       0.193789
4       0.054712
5       0.109234
6       0.063096
7       0.046583
8       0.028427
9       0.029279
10      0.044321
